# Snippets

This is a compilation of useful code snippets.

## Combine Overlapping Intervals in Pandas

Suppose we have the start and end points of several intervals. How do we combine them if the intervals overlap?

This solution is taken from https://stackoverflow.com/questions/57882621/efficient-merge-overlapping-intervals-in-same-pandas-dataframe-with-start-and-fi. Credit goes to users Dev Khadka and John Smith.

Take the following example.

In [32]:
import pandas as pd

intervals = pd.DataFrame({
    "start_point": [3, 5, 0, 10],
    "end_point": [8, 7, 1, 11],
})

The $[3, 6]$ and $[5, 7]$ intervals should be combined into $[3, 7]$. We will solve this by grouping the rows together if thier intervals overlap, then we select the minimum start point and maximum end point in each group.

Sort by the start point.

In [41]:
tmp = intervals.sort_values("start_point").reset_index(drop=True)
tmp

,start_point,end_point
0,0,1
1,3,8
2,5,7
3,10,11


Now, we compare start point $i$ with end point $i - 1$.

In [42]:
tmp["end_point"].shift()

0    NaN
1    1.0
2    8.0
3    7.0
Name: end_point, dtype: float64

We can use this to tell where the overlapping intervals are. For example, in rows 1 the start point 3 is greater than 1 so the $0$-th and $1$-st intervals do not overlap. However, in row 2 the end point 8 is greater than the start point 5 so the $1$-st and $2$-nd intervals overlap.

Row 3 interesting because the end point previous end point is greater than it. Since we sorted the start points, end an end point is less than its neighbour, we can guarantee it is contained within the interval. So, we include a `cummax`.

In [49]:
(tmp["start_point"] > tmp["end_point"].shift().cummax()).cumsum()

0    0
1    1
2    1
3    2
dtype: int64

Now we can group together the intervals which overlap.

In [53]:
groups = (tmp["start_point"] > tmp["end_point"].shift().cummax()).cumsum()
tmp.groupby(groups).agg({"start_point": "min", "end_point": "max"})

,start_point,end_point
0,0,1
1,3,8
2,10,11


Now we write this mathematically.

Suppose we have a set of intervals $X = \{ (l_1, r_1), (l_2, r_2), \dots, (l_n, r_n)\}$ that are sorted such that $l_1 \leq l_2 \leq \dots \leq l_n$. How do we find intervals that overlap and combine them?

This has two steps:

1. Group the intervals that overlap together.
2. Find the minimum of left intervals and maximum of the right invervals for each group.

Step 1 is the more difficult part. First not that if interval $i$ and $i + 1$ do not overlap then interval $i$ will not overlap with any interval $j$ where $j > i$. Since by definition $r_i < l_{i + 1} \leq l_{i + 2} \leq l_{i + 3} \leq ... $. This means that we can group the intervals from left to right and guarantee that none of the intervals in a group toward the left will overlap an interval toward the right.

We can find these groups of overlapping intervals by looking at interval $i$ and checking if it overlaps with interval $i + 1$. If it does not then it will not overlap with any intervals past $i + 1$ so interval $i$ and any of its preceeding overlapping intervals can be thought of as a group.

In [67]:
tmp = intervals.sort_values("start_point").reset_index(drop=True)

previous_end_point = tmp["end_point"].shift().cummax()
groups = (tmp["start_point"] > previous_end_point).cumsum()
tmp.groupby(groups).agg({"start_point": "min", "end_point": "max"})

SpecificationError: nested renamer is not supported